<a href="https://colab.research.google.com/github/dpostolovski/eeg_is/blob/train_compare_full_data/(-_-).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
  <img alt="FINKI **LOGO**" height="auto" src="https://www.finki.ukim.mk/Content/dataImages/downloads/logo-large-500x500_2.png" hspace="10px" vspace="0px">
  <h1>
    Интелигентни системи - Лабораториска вежба
  </h1>

  <h3><i>Група 5</i></h3>
  <h4><i>Дамjан Постоловски, Стефан Тодоровски, Ангела Кралевска, Иван Крстев, Мариjа Величковска</i></h2>
</center>

---



In [1]:
#@title Монтирање на Google Drive податочниот систем
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
#@title Инсталирање, вчитување и иницијализација на потребните библиотеки
!pip install mne 
!pip install termcolor

%tensorflow_version 1.12.0

import os
import numpy as np
import mne

from sklearn import metrics 
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.utils import to_categorical as to_cat
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from scipy.io import loadmat
from collections import Counter

%cd drive/My\ Drive/Интелигентни\ Системи
from EEGModels import DeepConvNet, EEGNet

!mkdir saved_models

K.set_image_data_format('channels_first')

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.12.0`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
[Errno 2] No such file or directory: 'drive/My Drive/Интелигентни Системи'
/content/drive/.shortcut-targets-by-id/1foXUZVE0mxnmx1CREGLvuWb3DEHpgrMB/Интелигентни Системи
mkdir: cannot create directory ‘saved_models’: File exists


## Иницијализација на функции кои ќе ги користиме
1. load_mat_data - ги вчитува .mat податоците.
2. load_txt_data - ги вчитува .txt податоците.
3. load_data - ги повикува претходните две функции и ги вчитува податоците за даден учесник.
4. reshape -
5. reshape_data_to_mne_format -





In [75]:
def load_mat_data(folder_path, file_name, mode = 'Train'):
    """
        Load and concatenate train mat data from all three sessions.

        :param folder_path: path to the participant data
        :param file_name: the name of the mat file to be loaded (without .mat extension)
        :param mode: Train or Test data 
        :return: numpy array containing the mat data
    """
    data = np.empty(0)
    for session in range(1, 4):  # Итерација низ секоја сесија
        path = folder_path + '/S' + format(session, '02') + '/' + mode + '/' + file_name + '.mat'
        temp = loadmat(path)[file_name]
        if data.size != 0:
            data = np.concatenate((data, temp), axis=2)
        else:
            data = np.array(temp)
    return data


def load_txt_data(folder_path, file_name, mode = 'Train'):
    """
        Load and concatenate txt data from all three sessions.

        :param folder_path: path to the participant data
        :param file_name: the name of the file to be loaded
        :param mode: Train or Test data 
        :return: numpy array containing the wanted data
    """
    data = np.empty(0)
    for session in range(1, 4):  # Итерација низ секоја сесија
        path = folder_path + '/S' + format(session, '02') + '/' + mode + '/' + file_name
        with open(path, "r") as file:
            temp = file.read().splitlines()
            if file_name == 'trainLabels.txt':
                temp = np.repeat(temp, 8 * 10)
            if data.size != 0:
                data = np.concatenate((data, temp))
            else:
                data = np.array(temp)
    return data


def load_data(participant):
    """
        Load data for the given participant.

        :param participant: Participant number (1-15)
        :return: train_data, train_labels, train_events, train_targets
    """
    print("Вчитување податоци од испитниот примерок (учесник) " + str(participant) + '...')
    path_to_data = 'Data/SBJ' + format(participant, '02')

    train_data = load_mat_data(path_to_data, 'trainData')
    train_labels = load_txt_data(path_to_data, 'trainLabels.txt')
    train_events = load_txt_data(path_to_data, 'trainEvents.txt')
    train_targets = load_txt_data(path_to_data, 'trainTargets.txt')

    print("Податоците од испитниот примерок (учесник) " + str(participant) + " се вчитани.\n")
    return train_data, train_labels, train_events, train_targets


def load_test_data(participant):
    """
        Load test data for the given participant.

        :param participant: Participant number (1-15)
        :return: test_date, test_events
    """
    print("Вчитување тест податоци од испитниот примерок (учесник) " + str(participant) + '...')
    path_to_data = 'Data/SBJ' + format(participant, '02')

    test_date = load_mat_data(path_to_data, 'testData', 'Test')
    test_events = load_txt_data(path_to_data, 'testEvents.txt', 'Test')

    print("Податоците од испитниот примерок (учесник) " + str(participant) + " се вчитани.\n")
    return test_date, test_events


def reshape(data, labels, events, targets):
  mne_array = np.swapaxes(data, 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0], 1, 8, 350)

  labels_arr = labels.astype(np.int)
  events_arr = events.astype(np.int)
  return mne_array, labels_arr-1, events_arr-1


def reshape_data_to_mne_format(data):
  mne_array = np.swapaxes(data, 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0], 1, 8, 350)

  return mne_array

In [70]:
PARTICIPANT_START = 5
PARTICIPANT_END = 7

# Глобална низа каде ќе се зачувуваат тренираните модели
models = []

for participant in range(PARTICIPANT_START, PARTICIPANT_END):  # Итерација низ секој испитен примерок
    train_data, train_labels, train_events, train_targets = load_data(participant)
    
    data, labels, events = reshape(train_data, train_labels, train_events, train_targets)
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

    model = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
    model.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
    checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                  verbose=1, save_best_only=True)

    y_train = to_cat(y_train)
    y_test = to_cat(y_test)

    num_batch_size=100
    num_epochs=50
    model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
              validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

    score = model.evaluate(X_test, y_test, verbose=1)
    print(score)

    models.append(model) # Ke imame 15 modeli neli, pa vo niza se staveni
                        # pristap do niv model[i]

Вчитување тест податоци од испитниот примерок (учесник) 5...
Податоците од испитниот примерок (учесник) 5 се вчитани.

Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/50
3400/3600 [===========================>..] - ETA: 0s - loss: 2.3218 - acc: 0.1526
Epoch 00001: val_loss improved from inf to 2.41387, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 13s 4ms/sample - loss: 2.3192 - acc: 0.1556 - val_loss: 2.4139 - val_acc: 0.1158
Epoch 2/50
3400/3600 [===========================>..] - ETA: 0s - loss: 2.2736 - acc: 0.1697
Epoch 00002: val_loss did not improve from 2.41387
3600/3600 [==============================] - 1s 280us/sample - loss: 2.2745 - acc: 0.1672 - val_loss: 2.6321 - val_acc: 0.0908
Epoch 3/50
3400/3600 [===========================>..] - ETA: 0s - loss: 2.2257 - acc: 0.1776
Epoch 00003: val_loss did not improve from 2.41387
3600/3600 [==

In [77]:
# Иницијализација на променливите каде ќе бидат вчитани тест податоците
test_data = []
test_events = []
test_runs_per_block = [[i for i in range(3)] for j in range(15)] # Covek, Sesija

start_i = PARTICIPANT_START
for participant in range(PARTICIPANT_START, PARTICIPANT_END): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({participant}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(participant) + "...")
  
  test_data, test_events = load_test_data(participant)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на бројот на runs 
    full_path = 'Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  array_i = i - start_i
      
  
  # =========================================================================

  print("SBJ" + str(format(i, '02')) + "| Test_data: " + str(test_data[array_i].shape)) # test_data to predict
  print("SBJ" + str(format(i, '02')) + "| Test_events: " + str(len(test_events[array_i]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i, '02')) + " / S" + str(format(j, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data)
  predictions = models[array_i].predict(to_predict_data)
  print("SBJ" + str(format(i, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()

      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (5) ======================
Вчитување тест податоци од испитниот примерок 5...
Вчитување тест податоци од испитниот примерок (учесник) 5...
Податоците од испитниот примерок (учесник) 5 се вчитани.

	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 6 се вчитани.

SBJ06| Test_data: (350, 8800)
SBJ06| Test_events: 1
SBJ06 / S01| Runs per block: 4
SBJ06 / S02| Runs per block: 7
SBJ06 / S03| Runs per block: 9
SBJ06| Predictions: 8800
============== Сесија (0) ==============
4,4,0,0,4,0,3,5,3,0,4,0,4,4,4,3,0,3,5,2,3,2,3,5,3,3,0,0,0,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,2,0,
============== Сесија (1) ==============
0,4,0,0,0,0,0,3,0,0,0,0,0,2,0,0,4,0,2,2,3,0,2,3,0,0,0,4,4,2,0,0,2,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,
============== Сесија (2) ==============
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Визуелизација на податоците

In [ ]:
#@title Локација на сензорите
#@markdown Може да се забележи дека каналите кои ги отчитуваме се наоѓаат во средниот и задниот дел на мозокот.
montage = mne.channels.make_standard_montage("standard_alphabetic")
montage.plot();

In [ ]:
#@title Histogram и зависности помеѓу 8те канали
from scipy import io
import matplotlib.pyplot as plt
t = io.loadmat('drive/My Drive/Интелигентни Системи/Data/SBJ01/S01-Train/trainData.mat')['trainData'] 
t[0].shape

fig, axs = plt.subplots(2)
# plot histogram
axs[0].hist(t[0][3])
axs[1].plot(range(0,350), t[0][2][0:350])

k = 0
fig, axs = plt.subplots(8,8)
for i in range(8):
  for y in range(8):
    axs[i,y].plot(t[i][0], t[y][0])

In [ ]:
#@title Визуелизација на EEG сигналите во рамки на еден настан (првиот настан).
fig, axs = plt.subplots(8)
swapped_matrix = np.swapaxes(data, 1, 2) # (канал, настан, епоха).

for channel in range(0, 8):
  axs[channel].set_title(ch_names[channel])
  for epoch in range(0, 350):
    y_arr = data[channel][epoch][0]
    axs[channel].plot(range(0,350), swapped_matrix[channel][0])

In [ ]:
#@title Визуелизација на EEG сигналите во првата,средната и последната епоха.
#@markdown 1. Визуелизација на EEG податоците по канали за првата, средната и последната епоха.
#@markdown 2. Power Spectrum Density на 175-тата епоха. Во оваа епоха 
#@markdown сигналите би биле најстабилни за објектот кој светка.

middle_epoch = mne_array[175]
first_epoch = mne_array[0]
last_epoch = mne_array[349]
raw_middle = mne.io.RawArray(middle_epoch, mne_info)
raw_first = mne.io.RawArray(first_epoch, mne_info)
raw_last = mne.io.RawArray(last_epoch, mne_info)
for epoch in range(0, 350): # Секоја епоха
  i = 0
  for event in range(0, 1600): # Настан
    
    np.array([1, 2, 3])
    i = i+1



print("\n")
print(colored('====== 1 ======', 'red'))
print(colored("Прва епоха од сите 1600 настани", 'blue'))
raw_first.plot(n_channels=8, scalings=dict(eeg=40), title='EEG Signals',
         show=True, block=True, color = dict(eeg='blue'))
print(colored("Средна епоха од сите 1600 настани", 'blue'))
raw_middle.plot(n_channels=8, scalings=dict(eeg=40), title='EEG Signals',
         show=True, block=True, color = dict(eeg='blue'))
print(colored("Последна епоха од сите 1600 настани", 'blue'))
raw_last.plot(n_channels=8, scalings=dict(eeg=40), title='EEG Signals',
         show=True, block=True, color = dict(eeg='blue'))
print("\n")
print(colored('====== 2 ======', 'red'))
raw_middle.plot_psd(area_mode='range', show=False, average=True);
raw_data.plot_image();

In [ ]:
#@title Визуелна споредба на EEG податоците при светкање на првиот и последниот објект  
#@markdown Во првиот случај вршиме споредба на EEG податоците кога првиот и последниот 
#@markdown објект светнал без разлика дали бил target, додека во вториот случај ги споредуваме
#@markdown EEG податоците за случаи само кога тие објекти биле target објекти.

# Извлекување на настаните каде светнал првиот објект.
first_object_events = [index for index, value in enumerate(events_arr) if value == '1']
first_object_eeg_data = np.zeros((8,350, len(first_object_events)))
for channel in range(0, 8): # Секој канал
  for epoch in range(0, 350): # Секоја епоха
    i = 0
    for event in first_object_events: # Настан
      first_object_eeg_data[channel][epoch][i] = data[channel][epoch][event]
      i = i+1

# Креирање на соодветна дводимензионална низа за mne библиотеката
mne_first_obj_array = np.swapaxes(first_object_eeg_data, 0, 1) # (епохa, канал, настан).
epoch = mne_first_obj_array[175] 
first_object_raw = mne.io.RawArray(epoch, mne_info)





# Извлекување на настаните каде светнал првиот објект и бил target.
first_object_events_target = [index for index, value in enumerate(events_arr) if value == '1']
for event_pos in first_object_events_target:
  if targets_arr[event_pos] == 1:
    continue # Продолжи
  else:
    first_object_events_target.remove(event_pos) # Избриши -> Објектот не е target
first_object_target_eeg_data = np.zeros((8,350, len(first_object_events_target)))
for channel in range(0, 8): # Секој канал
  for epoch in range(0, 350): # Секоја епоха
    i = 0
    for event in first_object_events_target: # Настан
      first_object_target_eeg_data[channel][epoch][i] = data[channel][epoch][event]
      i = i+1

# Креирање на соодветна дводимензионална низа за mne библиотеката
mne_first_obj_target_array = np.swapaxes(first_object_target_eeg_data, 0, 1) # (епохa, канал, настан).
epoch = mne_first_obj_target_array[175] 
first_object_target_raw = mne.io.RawArray(epoch, mne_info)





# Извлекување на настаните каде светнал последниот објект.
last_object_events = [index for index, value in enumerate(events_arr) if value == '8']
last_object_eeg_data = np.zeros((8,350, len(last_object_events)))
for channel in range(0, 8): # Секој канал
  for epoch in range(0, 350): # Секоја епоха
    i = 0
    for event in last_object_events: # Настан
      last_object_eeg_data[channel][epoch][i] = data[channel][epoch][event]
      i = i+1

# Креирање на соодветна дводимензионална низа за mne библиотеката
mne_last_obj_array = np.swapaxes(last_object_eeg_data, 0, 1) # (епохa, канал, настан).
epoch = mne_last_obj_array[175] 
last_object_raw = mne.io.RawArray(epoch, mne_info)





# Извлекување на настаните каде светнал последниот објект и бил target.
last_object_events_target = [index for index, value in enumerate(events_arr) if value == '8']
for event_pos in last_object_events_target:
  if targets_arr[event_pos] == 1:
    continue # Продолжи
  else:
    last_object_events_target.remove(event_pos) # Избриши -> Објектот не е target
last_object_target_eeg_data = np.zeros((8,350, len(last_object_events_target)))
for channel in range(0, 8): # Секој канал
  for epoch in range(0, 350): # Секоја епоха
    i = 0
    for event in last_object_events_target: # Настан
      last_object_target_eeg_data[channel][epoch][i] = data[channel][epoch][event]
      i = i+1
      
# Креирање на соодветна дводимензионална низа за mne библиотеката
mne_last_obj_target_array = np.swapaxes(last_object_target_eeg_data, 0, 1) # (епохa, канал, настан).
epoch = mne_last_obj_target_array[175] 
last_object_target_raw = mne.io.RawArray(epoch, mne_info)





# Исцртување и печатење
print("\n\n")
print(colored('====== ПРВ ОБЈЕКТ - Книги на полица ======', 'red'))
print(colored("Вкупно светнал: " + str(len(first_object_events)), 'blue'))
print(colored("Вкупно светнал и бил target: " + str(len(first_object_events_target)), 'blue'))
print("\n")
print(colored('====== ПОСЛЕДЕН ОБЈЕКТ - Слика закачена на ѕид ======', 'red'))
print(colored("Вкупно светнал: " + str(len(last_object_events)), 'blue'))
print(colored("Вкупно светнал и бил target: " + str(len(last_object_events_target)), 'blue'))
print("\n")
print(colored('====== Графици со EEG податоци кога светнале првиот и последниот објект  ======', 'green'))
first_object_raw.plot(n_channels=8, scalings=dict(eeg=40), title='EEG Signals',
         show=True, block=True, color = dict(eeg='blue'));
last_object_raw.plot(n_channels=8, scalings=dict(eeg=40), title='EEG Signals',
         show=True, block=True, color = dict(eeg='blue'));
print("\n")
print(colored('====== Графици со EEG податоци кога светнале првиот и последниот објект и биле target  ======', 'green'))
first_object_raw.plot(n_channels=8, scalings=dict(eeg=40), title='EEG Signals',
         show=True, block=True, color = dict(eeg='blue'));
last_object_raw.plot(n_channels=8, scalings=dict(eeg=40), title='EEG Signals',
         show=True, block=True, color = dict(eeg='blue'));

In [ ]:
from mne.preprocessing import ICA
raw.plot_psd(tmin=0.2, tmax=1.2, fmin=0, fmax=60, average=True, spatial_colors=False);
num_components = 8
ica = ICA(n_components=num_components, method='fastica')
ica.fit(raw)
ica.plot_sources(raw);

## Претпроцесирање

In [ ]:
#@title TO-DO
# TO-DO

In [ ]:
#@title Стандардизација на податоците

standardizer = mne.decoding.Scaler(scalings='mean')
standardizer.fit(mne_array)
standardized_data = standardizer.transform(mne_array)
print("Просекот е:",-round(standardized_data.mean()))
print("Варијансата е:",round(standardized_data.var()))

In [ ]:
#@title Нормализација метод бр. 2


import numpy as np
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
def min_max_scale(dataset: np.ndarray) -> np.ndarray:
    scaler = MinMaxScaler(feature_range=[0, 1])
    scaled_data = scaler.fit_transform(dataset)
    return scaled_data

def standard_scale(dataset: np.ndarray) -> np.ndarray:
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(dataset)
    return scaled_data

data = loadmat('drive/My Drive/Интелигентни Системи/Data/SBJ01/S01-Train/trainData.mat')['trainData'] 

v_min = data.min(axis=(0, 1), keepdims=True)
v_max = data.max(axis=(0, 1), keepdims=True)
print("Нормализирани податоци:")
(data - v_min)/(v_max - v_min)



In [ ]:
#@title Отстранување на шум - претстава

Min, Max = round(mne_array.min()),round(mne_array.max())*1000000
picks = mne.pick_types(raw_data.info, meg=False, eeg=True, stim=False, eog=False)
mne.viz.plot_epochs_image(raw_data, picks='eeg', vmin= Min, vmax=Max)


In [ ]:
#@title Отстранување на шум - реултати
xd = mne.preprocessing.Xdawn(n_components=2, signal_cov=None)
xd.fit(raw_data)
epochs_denoised = xd.apply(raw_data)
epochs_denoised.keys()
mne.viz.plot_epochs_image(epochs_denoised['1'], picks='eeg', vmin=Min, vmax=Max)

In [ ]:
#@title Downsampling
events = raw_data.events
epochs = mne.Epochs(raw, events, tmin=-0.1, tmax=1, preload=True)
print(epochs)
# Downsample to 175 Hz
print('Original sampling rate:', epochs.info['sfreq'], 'Hz')
epochs_resampled = epochs.copy().resample(175, npad='auto')
print('New sampling rate:', epochs_resampled.info['sfreq'], 'Hz')

# Plot a piece of data to see the effects of downsampling
plt.figure(figsize=(7, 3))

n_samples_to_plot = int(0.5 * epochs.info['sfreq'])  # plot 0.5 seconds of data
plt.plot(epochs.times[:n_samples_to_plot],
         epochs.get_data()[ 0, 0, :n_samples_to_plot], color='black')

n_samples_to_plot = int(0.5 * epochs_resampled.info['sfreq'])
plt.plot(epochs_resampled.times[:n_samples_to_plot],
         epochs_resampled.get_data()[ 0, 0, :n_samples_to_plot],
         '-o', color='red')

plt.xlabel('time (s)')
plt.legend(['original', 'downsampled'], loc='best')
plt.title('Effect of downsampling')
mne.viz.tight_layout()

In [ ]:
#@title Principal Component Analysis
pca = mne.decoding.UnsupervisedSpatialFilter(PCA(8), average=False)
pca_data = pca.fit_transform(mne_array)
ev = mne.EvokedArray(np.mean(pca_data, axis=0),
                     mne.create_info(8, raw_data.info['sfreq'],
                                     ch_types='eeg'), tmin=Min)
ev.plot(show=False, window_title="PCA", time_unit='ms')